# (1) Data collection using both, Web Scraping and a Web API.

### Overpass turbo query to get all available electronc stores in Switzerland

In [37]:
#Libaries
import os
import re
import time
import fnmatch
import numpy as np
import pandas as pd
from pandas import json_normalize
import requests
import json
!pip install overpy
import overpy
import matplotlib.pyplot as plt
import folium

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [38]:
# Overpass API URL
url = "http://overpass-api.de/api/interpreter"

# Overpass turbo query

query = f"""[out:json][timeout:25];
        area["ISO3166-1"="CH"][admin_level=2];
        (node["shop"="electronics"](area);
          way["shop"="electronics"](area);
          relation["shop"="electronics"](area);
        );
        out body;
        >;
        out skel qt;"""


# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']


# Save data to file
with open('amazon_laptops.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)
df.head(10)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# Mit Python auf Overpass-API zugreifen, indem man das overpy-Paket als "Wrapper" verwendet
api = overpy.Overpass()
r = api.query("""
        area["ISO3166-1"="CH"][admin_level=2];
        (node["shop"="electronics"](area);
        way["shop"="electronics"](area);
        relation["shop"="electronics"](area););
        out center; 
        """)
# Die Anweisung "out center" erstellt Mittelpunktskoordinaten unter dem Schlüssel center; 
# wenn Knotenelement, dann sind Koordinaten unter den Schlüsseln Iat und Ion zu finden
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

    
# Koordinaten in ein numpy array konvertieren
X = np.array(coords)
plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('Elektronikfachgeschäfte in der Schweiz')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

In [ ]:
# Subset of electronic stores by brand
locations = df[["lat", "lon", "tags.brand", "tags.shop"]].loc[(df["tags.brand"] == 'Interdiscount') | 
                                                              (df["tags.brand"] == 'Fust') |
                                                              (df["tags.brand"] == 'melectronics') |
                                                              (df["tags.brand"] == 'Media Markt ') |
                                                               (df["tags.brand"] == 'Digitec') 
                                                              
                                                             ] 
print(locations.head(20))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.brand"]).add_to(map)

# Plot map
map

# (2) Data preparation (e.g. remove missing values and duplicates, create new variables, enrich the data with open data).

### Importing data

In [ ]:
# Get current working directory
print(os.getcwd())

# Show all files in the directory
flist = fnmatch.filter(os.listdir('.'), '*.csv')
for i in flist:
    print(i)

# Read the data to a pandas data frame
df = pd.read_csv('amazon_laptops.csv', sep=',', encoding='utf-8')

In [ ]:
## Count rows and columns
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns
print('Number of columns:', df.shape[1])

In [ ]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

In [ ]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values
df[df.isna().any(axis=1)]

In [ ]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['web-scraper-order', 'price_raw', 'title_raw']].duplicated()]

In [ ]:
# Extract values from 'price_raw' strings
price = []
for x in df['price_raw']:
    d1 = re.findall('[0-9]+', str(x))
    try:
        d2 = d1[0].strip()
    except:
        d2 = None
    price.append(d2)

# Save as new variable in the pandas data frame
df['price'] = pd.Series(price, dtype="Int64")

# Print first 5 values
print(df['price_raw'].head(5), '\n')
print(df['price'].head(5))

In [ ]:
# Extract values from 'dimensions_raw' strings
dimension = []
for i in df['dimensions_raw']:
    d1 = re.findall('(.*)cm', str(i))
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    dimension.append(d2)

# Save as new variable in the pandas data frame
df['dimension'] = pd.Series(dimension, dtype="string")
    
# Print first 5 values
print(df['dimensions_raw'].head(5), '\n')
print(df['dimension'].head(5))

In [ ]:
# Extract values from 'dissize_raw' strings

displaysize = []
for i in df['dissize_raw']:
    d1 = re.findall('(.*)Zoll', str(i))
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    displaysize.append(d2)

# Save as new variable in the pandas data frame
df['displaysize'] = pd.Series(displaysize, dtype="string")
    
# Print first 5 values
print(df['dissize_raw'].head(5), '\n')
print(df['displaysize'].head(5))

In [ ]:
df.to_csv('amazon_laptopslist_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

#### (3) Data storage in a database like sqlite or MySQL.

In [47]:
## Create sqlite database
import pandas as pd
import sqlite3
import fnmatch
import os

#Get current working directory
print(os.getcwd())

#Create a database
conn = sqlite3.connect('amazon_laptops.db')
cursor = conn.cursor()

#Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'),'*.db')
for i in flist:
    print(i)

C:\Workspace_ZHAW\DA\projectwork
amazon_laptops.db


In [48]:
## Create SQL-table in the database
cursor.execute('''CREATE TABLE IF NOT EXISTS amazonlaptops_table (OrderId VARCHAR(50), Dimensions DECIMAL (8,2), dissize INT(8),Price DECIMAL (8,2))''')

#Confirm change to the table
conn.commit()

In [49]:
## Read data from file to dataframe
df = pd.read_csv('amazon_laptopslist_prepared.csv',
                sep = ',',
                encoding = 'utf-8')[['web-scraper-order', 'dimension', 'displaysize','price']]
print(df.shape)
df.head(5)

(120, 4)


,web-scraper-order,dimension,displaysize,price
0,1669994182-1,‎32.42 x 21.57 x 1.99,‎15.6,659.0
1,1669994186-2,‎30.8 x 22.3 x 1.45,‎13.5,1.0
2,1669994190-3,‎25.39 x 35.78 x 1.87,‎16,1.0
3,1669994194-4,‎28.5 x 19.5 x 1.95,‎10.1,187.0
4,1669994198-5,‎35.4 x 25.1 x 2.24,‎15.6,965.0


In [50]:
## Write data to the SQL-table in database
df.to_sql(name = 'amazonlaptops_table',
         con = conn,
         index = False,
         if_exists = 'replace')

120

In [54]:
## Query the SQL-Table

cursor.execute('''SELECT * FROM amazonlaptops_table
WHERE displaysize >= 15
AND Price > 1100''')

df = pd.DataFrame(cursor.fetchall(),
                 columns = ['OrderId', 'Dimensions','Displaysize','Price'])

print(df)

Empty DataFrame
Columns: [OrderId, Dimensions, Displaysize, Price]
Index: []


In [55]:
## Plot histogram of ""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df.Price.plot.hist(grid = True,
                  bins = 20,
                  rwidth = 0.9,
                  color = '#607c8e',)
plt.title('Laptop Price')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha = 0.75)

conn.close()

TypeError: no numeric data to plot

#### (4) Non-graphical and graphical exploratory data analysis (EDA).
#### (5) Use of either regression or classification as the modelling method.
#### (6) Model evaluation using suitable measures of fit.
#### (7) Correct interpretation of model results and measures of fit.
#### (8) Making the material (data, Jupyter notebooks, ...) available on Moodle.

#### 1) Creativity of implementation (creative is anything not specified in the lessons and exercises).
#### (2) Use of a MySQL database for data storage and SQL-queries from within Python.
#### (3) Integration and visualization of geographical data.
#### (4) Use of a statistical test for the analysis of contingency tables or analysis of variance (ANOVA).
#### (5) Use of k-means clustering in addition to the regression or classification model.

In [ ]:
## footer info
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')